## Вручтель Серафима, 495 группа, Контест на прогнозирование спроса.

# Большая просьба: не читайте, пожалуйста, что написано в этом ноутбуке до слова ОТЧЁТ, тут местами полнейший треш.

In [249]:
import pandas as pd

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [250]:
frac = 0.1 # fraction of learning examples used for model fitting
#frac = 1

# sample the train set if your don't want to deel with all examples
train = train.sample(frac=frac, random_state=42)

X = train.drop(['Num','y'], axis=1)
y = train['y']
print(len(X), len(y))

7246 7246


In [251]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

#model = GradientBoostingRegressor(n_estimators=100, max_depth=12, random_state=42)
model = RandomForestRegressor(n_estimators=100)
model.fit(X, y)

preds = model.predict(test.drop(['Num'], axis=1))

print(len(preds))
print(len(sample_submission))

2016
2016
CPU times: user 52.6 s, sys: 0 ns, total: 52.6 s
Wall time: 52.6 s


In [252]:
sample_submission['y'] = preds

In [253]:
sample_submission.head(5)

,Num,y
0,348622,1667.35
1,348623,26811.91
2,348624,314262.51
3,348625,31135.05
4,348626,370.22


In [254]:
# In GBM you can get some negative predictions:
print(sample_submission[sample_submission['y'] < 0])

Empty DataFrame
Columns: [Num, y]
Index: []


In [255]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [256]:
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)

In [34]:
# Попробуем замерить качество используемой модели

from sklearn.cross_validation import cross_val_score


In [36]:
%%time
# Кросс валидация при использовании градиентного бустинга
print(cross_val_score(model, X, y).mean())

0.903507626754
CPU times: user 5min 19s, sys: 40 ms, total: 5min 19s
Wall time: 5min 19s


## Random Forest

In [37]:
from sklearn.ensemble import RandomForestRegressor

In [38]:
f = X.shape[1]

In [28]:
%%time
model_random_forest_0 = RandomForestRegressor(n_estimators=100, max_depth=10, max_features=int(f ** 0.5))
print(cross_val_score(model_random_forest_0, X, y).mean())
#model_random_forest_0.fit(X, y)

0.927510030851
CPU times: user 9.08 s, sys: 0 ns, total: 9.08 s
Wall time: 9.09 s


In [32]:
# Попробуем разную максимальную глубину
depths = range(2, 15)
for M_depth in depths:
    model_random_forest_cur = RandomForestRegressor(n_estimators=100, max_depth=M_depth, max_features=int(f ** 0.5))
    print("Max depth: ", M_depth, "cross_val_score: ", cross_val_score(model_random_forest_cur, X, y).mean())


Max depth:  2 cross_val_score:  0.891900537847
Max depth:  3 cross_val_score:  0.921275398746
Max depth:  4 cross_val_score:  0.926086233452
Max depth:  5 cross_val_score:  0.924395333021
Max depth:  6 cross_val_score:  0.926980907453
Max depth:  7 cross_val_score:  0.927975809638
Max depth:  8 cross_val_score:  0.926645861674
Max depth:  9 cross_val_score:  0.926649285929
Max depth:  10 cross_val_score:  0.927263402546
Max depth:  11 cross_val_score:  0.926555667174
Max depth:  12 cross_val_score:  0.926313914946
Max depth:  13 cross_val_score:  0.927043466825
Max depth:  14 cross_val_score:  0.924655621446


Наилучшие результаты получились при max_depth равном 7, 10 и 13.

In [34]:
# Попробуем разное количество деревьев

import numpy as np

trees_num = np.arange(10, 250, 10)
for trees_n in trees_num:
    model_random_forest_cur = RandomForestRegressor(n_estimators=trees_n, max_depth=10, max_features=int(f ** 0.5))
    print("Trees number: ", trees_n, "cross_val_score: ", cross_val_score(model_random_forest_cur, X, y).mean())


Trees number:  10 cross_val_score:  0.914681780287
Trees number:  20 cross_val_score:  0.921584786635
Trees number:  30 cross_val_score:  0.925689132517
Trees number:  40 cross_val_score:  0.925455078442
Trees number:  50 cross_val_score:  0.924764109672
Trees number:  60 cross_val_score:  0.926765033074
Trees number:  70 cross_val_score:  0.928427546975
Trees number:  80 cross_val_score:  0.928654157052
Trees number:  90 cross_val_score:  0.926374437468
Trees number:  100 cross_val_score:  0.92671830565
Trees number:  110 cross_val_score:  0.926638647829
Trees number:  120 cross_val_score:  0.928000699942
Trees number:  130 cross_val_score:  0.925724821795
Trees number:  140 cross_val_score:  0.926141615138
Trees number:  150 cross_val_score:  0.927238883589
Trees number:  160 cross_val_score:  0.927641368519
Trees number:  170 cross_val_score:  0.928273085445
Trees number:  180 cross_val_score:  0.927820000096
Trees number:  190 cross_val_score:  0.926205367811
Trees number:  200 cro

Наилучшие результаты получились на следующих количествах деревьев: 70, 80, 120, 170

Теперь исследуем подробнее варианты с лучшими количествами деревьев и лучшей максимальной глубиной (понятно, что рандомизация в сплитах накладывает отпечаток, но нужно же из чего-то выбирать, пусть будет такая вот эвристика).

In [37]:
best_max_depths = [7, 10, 13]
best_trees_num = [70, 80, 120, 170]

for M_depth in best_max_depths:
    print("Max depth: ", M_depth)
    for trees_n in best_trees_num:
        model_random_forest_cur = RandomForestRegressor(n_estimators=trees_n, 
                                                        max_depth=M_depth, 
                                                        max_features=int(f ** 0.5))
        print("\tTrees number: ", trees_n, "cross_val_score: ", cross_val_score(model_random_forest_cur, X, y).mean())
    print("")        
        

Max depth:  7
	Trees number:  70 cross_val_score:  0.925267729012
	Trees number:  80 cross_val_score:  0.926784551085
	Trees number:  120 cross_val_score:  0.92755375031
	Trees number:  170 cross_val_score:  0.927402085815

Max depth:  10
	Trees number:  70 cross_val_score:  0.925550546325
	Trees number:  80 cross_val_score:  0.925178579232
	Trees number:  120 cross_val_score:  0.926860209309
	Trees number:  170 cross_val_score:  0.927677689606

Max depth:  13
	Trees number:  70 cross_val_score:  0.92594808121
	Trees number:  80 cross_val_score:  0.924943057423
	Trees number:  120 cross_val_score:  0.928122289938
	Trees number:  170 cross_val_score:  0.928313355779



Получили наилучшие результат при максимальной глубине деревьев 13 и максимальном количестве деревьев 170, такую модель и попробуем использовать.

## Важно: запустить ещё раз и попробовать залить ЭТОТ результат.

In [45]:
%%time
train = pd.read_csv("train.tsv")
frac = 0.1
train = train.sample(frac=frac, random_state=42)

X = train.drop(['Num','y'], axis=1)
y = train['y']
print(len(X), len(y))

model_random_forest = RandomForestRegressor(n_estimators=170, max_depth=13, max_features=int(f ** 0.5))
model_random_forest.fit(X, y)
preds_rf = model_random_forest.predict(test.drop(['Num'], axis=1))

print(len(preds_rf))
print(len(sample_submission))

sample_submission['y'] = preds_rf
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

sample_submission.to_csv("sample_submission.tsv", sep=',', index=False)


7246 7246
2016
2016
CPU times: user 10.5 s, sys: 36 ms, total: 10.6 s
Wall time: 10.6 s


### Попробуем осуществить предобработку данных

In [46]:
from sklearn.preprocessing import scale

X_scaled = scale(X)


In [48]:
%%time
# Посмотрим, как изменится результат на новых данных
print("Cross val with latest random forest without scaling: ", cross_val_score(model_random_forest, X, y).mean())
print("With scaling: ", cross_val_score(model_random_forest, X_scaled, y).mean())


Cross val with latest random forest without scaling:  0.926974218349
With scaling:  0.925780752861
CPU times: user 36.9 s, sys: 20 ms, total: 36.9 s
Wall time: 36.9 s


Результат отрицательный :)

Можно было бы попробовать ещё выбросы убрать, может быть потом...

(при перезапуске ячеек дало положительный результат: в общем, попробовать можно, но вряд ли это даст ощутимый эффект)

Попробуем ещё немного обработать данные.


In [57]:
train = pd.read_csv("train.tsv")
frac = 0.1
train = train.sample(frac=frac, random_state=42)

X = train.drop(['Num','y'], axis=1)
y = train['y']
print(len(X), len(y))


7246 7246


In [58]:
X.head()

,year,week,shift,item_id,f1,f2,f3,f4,f5,f6,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
44283,2014,13,1,20452327,129441.0,104610.0,121114.0,133780.0,122580.0,126830.0,...,95880.0,88520.0,107270.0,79610.0,99210.0,114561.0,93790.0,98070.0,83980.0,105240.0
50871,2014,23,1,20441989,4162.0,6760.0,7210.0,11330.0,6950.0,6798.0,...,9932.0,8908.0,10464.0,7431.0,10334.0,11548.0,8698.0,8696.0,8160.0,7570.0
13810,2013,21,3,20438706,24931.0,30338.0,30690.0,37930.0,21420.0,28240.0,...,34475.0,22390.0,27124.0,29660.0,30105.0,28054.0,31545.0,28185.0,34890.0,28790.0
10062,2013,15,2,20438591,11505.0,13550.0,15360.0,14750.0,12961.0,9880.0,...,10920.0,14030.0,8060.0,8430.0,9980.0,13930.0,6340.0,7810.0,8960.0,10260.0
37186,2014,3,2,20449525,0.0,0.0,0.0,0.0,0.0,0.0,...,2090.0,2470.0,1145.0,2955.0,3915.0,740.0,2260.0,1403.0,1417.0,980.0


In [59]:
X = X.drop(['item_id'], axis=1)
X.head()


,year,week,shift,f1,f2,f3,f4,f5,f6,f7,...,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60
44283,2014,13,1,129441.0,104610.0,121114.0,133780.0,122580.0,126830.0,102220.0,...,95880.0,88520.0,107270.0,79610.0,99210.0,114561.0,93790.0,98070.0,83980.0,105240.0
50871,2014,23,1,4162.0,6760.0,7210.0,11330.0,6950.0,6798.0,9470.0,...,9932.0,8908.0,10464.0,7431.0,10334.0,11548.0,8698.0,8696.0,8160.0,7570.0
13810,2013,21,3,24931.0,30338.0,30690.0,37930.0,21420.0,28240.0,28685.0,...,34475.0,22390.0,27124.0,29660.0,30105.0,28054.0,31545.0,28185.0,34890.0,28790.0
10062,2013,15,2,11505.0,13550.0,15360.0,14750.0,12961.0,9880.0,11950.0,...,10920.0,14030.0,8060.0,8430.0,9980.0,13930.0,6340.0,7810.0,8960.0,10260.0
37186,2014,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2090.0,2470.0,1145.0,2955.0,3915.0,740.0,2260.0,1403.0,1417.0,980.0


In [60]:
X = X.drop(['f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44'], axis=1)

In [61]:
X = X.drop(['f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58'], axis=1)

In [62]:
X = X.drop(['f59', 'f60'], axis=1)
X.head()


,year,week,shift,f1,f2,f3,f4,f5,f6,f7,...,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30
44283,2014,13,1,129441.0,104610.0,121114.0,133780.0,122580.0,126830.0,102220.0,...,95880.0,88520.0,107270.0,79610.0,99210.0,114561.0,93790.0,98070.0,83980.0,105240.0
50871,2014,23,1,4162.0,6760.0,7210.0,11330.0,6950.0,6798.0,9470.0,...,9932.0,8908.0,10464.0,7431.0,10334.0,11548.0,8698.0,8696.0,8160.0,7570.0
13810,2013,21,3,24931.0,30338.0,30690.0,37930.0,21420.0,28240.0,28685.0,...,34475.0,22390.0,27124.0,29660.0,30105.0,28054.0,31545.0,28185.0,34890.0,28790.0
10062,2013,15,2,11505.0,13550.0,15360.0,14750.0,12961.0,9880.0,11950.0,...,10920.0,14030.0,8060.0,8430.0,9980.0,13930.0,6340.0,7810.0,8960.0,10260.0
37186,2014,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2090.0,2470.0,1145.0,2955.0,3915.0,740.0,2260.0,1403.0,1417.0,980.0


In [75]:
%%time

f = X.shape[1]
model_rf_min_data = RandomForestRegressor(n_estimators=170, max_depth=13, max_features=int(f ** 0.5))
model_rf_min_data.fit(X, y)

print("With min data: ", cross_val_score(model_rf_min_data, X, y).mean())


With min data:  0.927848124722
CPU times: user 17.5 s, sys: 4 ms, total: 17.5 s
Wall time: 17.5 s


Почти не повлияло, но пусть уж так.

In [71]:
test = pd.read_csv("test.tsv")
test_min_data = test.drop(['Num'], axis=1)
test_min_data = test_min_data.drop(['item_id'], axis=1)
test_min_data = test_min_data.drop(['f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39',
                           'f40', 'f41', 'f42', 'f43', 'f44',
                           'f45', 'f46', 'f47', 'f48', 'f49',
                           'f50', 'f51', 'f52', 'f53', 'f54',
                           'f55', 'f56', 'f57', 'f58', 'f59', 
                           'f60'], axis=1)


In [73]:
preds_rf_min_data = model_rf_min_data.predict(test_min_data)

print(len(preds_rf_min_data))
print(len(sample_submission))

sample_submission['y'] = preds_rf_min_data
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

sample_submission.to_csv("sample_submission.tsv", sep=',', index=False)


2016
2016


In [247]:
# Что-то ну уж совсем странное
X_strange = X.drop(['f1',
                    'f2',
                    'f3',
                    'f4',
                    'f5',
                    'f6',
                    #'f7',
                    'f8',
                    'f9',
                    #'f10',
                    #'f11',
                    'f12',
                    'f13',
                    #'f14',
                    'f15',
                    'f16',
                    'f17',
                    'f18',
                    'f19',
                    'f20',
                    'f21',
                    #'f22',
                    #'f23',
                    'f24',
                    'f25',
                    'f26',
                    #'f27',
                    #'f28',
                    #'f29',
                    'f30',
                    ], axis=1)

f = X_strange.shape[1]
model_rf_strange = RandomForestRegressor(n_estimators=170, max_depth=13, max_features=int(f ** 0.5))
model_rf_strange.fit(X_strange, y)

print("With strange data: ", cross_val_score(model_rf_min_data, X_strange, y).mean())


With strange data:  0.938225423346


In [240]:
test_strange = test_min_data.drop(['f1','f2', 'f3', 'f4', 'f5', 'f6', 'f8', 'f9', 'f12', 'f13', 'f15',
                                   'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f24', 'f25', 'f26', 'f30'], axis=1)

preds_rf_strange = model_rf_strange.predict(test_strange)

print(len(preds_rf_strange))
print(len(sample_submission))

sample_submission['y'] = preds_rf_strange
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

sample_submission.to_csv("sample_submission.tsv", sep=',', index=False)


2016
2016


# ОТЧЁТ:

***Краткая формулировка задачи:***

Даны предпосчитанные средние значения продаж конкретных товаров по разным временным срезам.

Требуется написать алгоритм машинного обучения, прогнозирующий по историческим данным о продажах количество проданных товаров.

***Наилучшим оказалось следующее решение:***

In [257]:
%%time

import pandas as pd
from sklearn.ensemble import RandomForestRegressor

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

frac = 0.1

train = train.sample(frac=frac, random_state=42)

X = train.drop(['Num','y'], axis=1)
y = train['y']

model = RandomForestRegressor(n_estimators=100)
model.fit(X, y)

preds = model.predict(test.drop(['Num'], axis=1))

sample_submission['y'] = preds

sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

sample_submission.to_csv("sample_submission.tsv", sep=',', index=False)


CPU times: user 55.2 s, sys: 84 ms, total: 55.2 s
Wall time: 55.2 s


Здесь отсутствует какая-либо существенная предобработка данных, обучается просто RandomForestRegressor на 100 деревьях.

***Другие подходы:***

Пробовала также градиентный бустинг, но основной исследуемой моделью был RandomForest.

Исследовала качество на RandomForest с различным количеством деревьев и различной глубиной этих деревьев. Качество SMAPE на наилучшем из полученных RandomForest оказалось лучше чем с использованием градиентного бустинга.

Также пыталась предобрабатывать данные. Сначала я их отскалировала, но существенного результата это не дало, что и понятно, скалирование как-то существенно может сказаться на линейных моделях, коей RandomForest не является. Кроме того, я удалила категориальный признак item_id, и пыталась понять, какие ещё признаки коррелируют и поэтому их не стоит учитывать в построении модели. Желаемого эффекта это не дало: на SMAPE с точностью до десятых долей не повлияло вообще.

***Качество*** определяла с помощью кросс-валидации с метрикой MSE, значения полученные при такой оценке качества с небольшой погрешностью коррелируют с метрикой SMAPE.